In [15]:
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv() # automatic find 

# load your api key as environment variables
load_dotenv(env_path)

True

In [16]:
!ls ../raw_data

Artist_studies_master.csv batch 3                   batch 6
batch 1                   batch 3 2                 batch 6 2
batch 1.zip               batch 4                   batch 7
batch 2                   batch 4 2
batch 2 2                 batch 5


In [17]:
import pandas as pd
studies_csv = pd.read_csv('../raw_data/Artist_studies_master.csv')
studies_df = studies_csv.copy()

In [18]:
studies_df.head()

,Artist (last/only)\n(or other effect),Artist (first),Style represented,Complete,User,sgl img folder,cards folder,Batch #,Check x if non-visual artist,Check x if style or other effect,Notes
0,Adams,Ansel,Yes,x,KyrickYoung,x,x,2.0,-,-,NaN
1,Adams,Arthur,Yes,x,sureailabs,x,x,5.0,-,-,NaN
2,Aenami,Alena,Yes,x,sureailabs,x,x,3.0,-,-,NaN
3,af Klint,Hilma,Yes,x,proximasan,x,x,1.0,-,-,NaN
4,Affandi,NaN,Yes,x,sureailabs,x,x,3.0,-,-,NaN


In [19]:
studies_df.columns

Index(['Artist (last/only)\n(or other effect)', 'Artist (first)',
       'Style represented', 'Complete', 'User', 'sgl img folder',
       'cards folder', 'Batch #', 'Check x if non-visual artist',
       'Check x if style or other effect', 'Notes'],
      dtype='object')

In [20]:
studies_df.columns = ['l_name', 'f_name',
       'style_represented', 'complete', 'user', 'sgl_img_folder',
       'cards_folder', 'batch_id', 'non_visual',
       'style_or_effect', 'notes']

In [21]:
# filter out styles and non visual artists
artist_df = studies_df[studies_df['complete'].str.contains('x') &
                       ~studies_df['non_visual'].str.contains('x') &
                       ~studies_df['style_or_effect'].str.contains('x')]


In [22]:
artists_interest_df = artist_df[artist_df['batch_id'] == 1.0]
len(artists_interest_df) * 0.05

7.1000000000000005

In [23]:
artists_interest_df.loc[:, 'key'] = artists_interest_df.apply(lambda row: f"{row['f_name']} {row['l_name']}", axis = 1).str.replace(' ', '_')

/Users/alecsharp/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [25]:
artists_interest_df

,l_name,f_name,style_represented,complete,user,sgl_img_folder,cards_folder,batch_id,non_visual,style_or_effect,notes,key
3,af Klint,Hilma,Yes,x,proximasan,x,x,1.0,-,-,NaN,Hilma_af_Klint
6,Aivazovsky,Ivan,Yes,x,KyrickYoung,x,x,1.0,-,-,NaN,Ivan_Aivazovsky
8,Amano,Yoshitaka,Yes,x,EErratica,x,x,1.0,-,-,NaN,Yoshitaka_Amano
10,Anderson,Wes,Yes,x,EErratica,x,x,1.0,-,-,NaN,Wes_Anderson
11,Andrews,Esao,Yes,x,sureailabs,x,x,1.0,-,-,NaN,Esao_Andrews
...,...,...,...,...,...,...,...,...,...,...,...,...
1088,Orbik,Glen,No,x,KyrickYoung,x,x,1.0,-,-,NaN,Glen_Orbik
1095,Parker,Lisa,No,x,KyrickYoung,x,x,1.0,-,-,NaN,Lisa_Parker
1127,Siudmak,Wojciech,No,x,sureailabs,x,x,1.0,-,-,NaN,Wojciech_Siudmak
1137,Szukalski,Stanisław,No,x,sureailabs,x,x,1.0,-,-,NaN,Stanisław__Szukalski


In [72]:
import wikipediaapi
import numpy as np
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

wiki_wiki = wikipediaapi.Wikipedia('en')

##### use regex for replacements #####
empty_row = "|------|--------|------------------|-----------------|--------|----------|-------------|-------------|-------------|"
empty_row2 = "| --- | --- | --- | --- | --- | --- | --- | --- | --- |"
empty_row3 = '|------|--------|------------------|-----------------|---------|----------|-------------|-------------|-------------|'

table = "| Name | Gender | Born (dd-mm-yyyy) | Died (dd-mm-yyyy) | Style | Location | Nationality | Occupation | Art School |"

headers = [x.strip() for x in table.split('|')[1:-1]]


In [28]:
# https://stackoverflow.com/questions/51122413/regex-for-extracting-all-complex-dates-formats-from-a-string-in-python
import re
import calendar

class DateRegex:
    
    def __init__(self):
        self.hello = 'hi'
        # create regex helper strings
        full_months = [month for month in calendar.month_name if month]
        short_months = [d[:3] for d in full_months]
        months = '|'.join(short_months + full_months)
        day = r'\d+'
        year = r'\d+'
        day_or_year = r'\d+(?:\w+)?'
        separator = r'[.,]?\s+'
        # compile regex
        self.regex_expression = re.compile(rf'(?:{day}{separator})?(?:{months}){separator}{day_or_year}(?:{separator}{year})?')
        
    def get_all_dates(self, input_text):
        
        return self.regex_expression.findall(input_text)

date_regex = DateRegex()

In [64]:
# estimate cost for all artists
round(len(artists_interest_df) * 0.05, 2)

7.1

In [73]:
from tqdm import tqdm
#records_list = []
#artist = 'Hubert_Robert'
#artist = 'Annibale_Carracci'

for artist in tqdm(artists_interest_df['key'][56:]):
    wiki_page = wiki_wiki.page(artist)
    if wiki_page.exists():

        artist_text = wiki_page.text
        
        print(artist_text[:50])

        prompt = "A table summarizing an Artist:\n\n" + artist_text[:2000] + "\n\n" + table


        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt = prompt,
            temperature=0,
            max_tokens=200,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )

        
        ##### use regex for replacements #####
        results = [x.strip() for x in response.choices[0].text.replace(empty_row, '').replace(empty_row2, '').replace(empty_row3, '').split('|')[1:-1]]

        results_dict = {k:v for k, v in zip(headers, results)}

        results_dict['Artist'] = artist
        results_dict['URL'] = wiki_page.fullurl

        records_list.append(results_dict)

  0%|                                                    | 0/86 [00:00<?, ?it/s]

Magdalena Carmen Frida Kahlo y Calderón (Spanish p


  1%|▌                                           | 1/86 [00:06<09:03,  6.39s/it]

Wassily Wassilyevich Kandinsky (; Russian: Василий


  2%|█                                           | 2/86 [00:10<07:24,  5.29s/it]

...

 97%|█████████████████████████████████████████▌ | 83/86 [05:36<00:14,  4.77s/it]

Wojciech Kazimierz "Wojtek" Siudmak (born 10 Octob


 98%|██████████████████████████████████████████ | 84/86 [05:40<00:09,  4.65s/it]

Stanisław Szukalski (13 December 1893 – 19 May 198


100%|███████████████████████████████████████████| 86/86 [05:45<00:00,  4.02s/it]


In [75]:
results_df = pd.DataFrame(records_list)

In [79]:
#results_df.to_csv('../data/batch_1_artist_info.csv')